# Logical Transduction

Agentics objects are capable of performing logical transduction between their states by using the provied LLMs. The logical transduction operator << can be applied between any two agentics of any type.

Transduction is done between a source and a target AG when connected by the << operator.

The follwing example transduces a Question into an Answer.

In [1]:
! uv pip install agentics-py


import os
import sys

from dotenv import find_dotenv, load_dotenv

CURRENT_PATH = ""

IN_COLAB = "google.colab" in sys.modules
print("In Colab:", IN_COLAB)

IN_COLAB = "google.colab" in sys.modules
print("In Colab:", IN_COLAB)

if IN_COLAB:
    CURRENT_PATH = "/content/drive/MyDrive/"
    from google.colab import drive
    drive.mount("/content/drive")
    load_dotenv("/content/drive/MyDrive/.env")
else:
    load_dotenv(find_dotenv())

if not os.getenv("GEMINI_API_KEY"):
    os.environ["GEMINI_API_KEY"] = input("Enter your GEMINI_API_KEY:")

Using Python 3.12.9 environment at: /Users/gliozzo/Code/agentics911/agentics/.venv
Audited 1 package in 21ms
In Colab: False
In Colab: False


In [ ]:
from agentics import Agentics as AG
from pydantic import BaseModel
from typing import Optional
from agentics.core.llm_connections import get_llm_providers


## Define target and source types
class Answer(BaseModel):
    answer: Optional[str] = None
    justification: Optional[str] = None
    confidence: Optional[float] = None

class Question(BaseModel):
    question :Optional[str] = None


## Instantiate the source AG with a question
source = AG(atype=Question,
            llm=get_llm_provider(), ## You can choose between "openai" (i.e. get_llm_provider("openai")), "watsonx", "gemini", "vllm_crewai"
                                    ## set verbose to true to see the internal agents log. This is optional.
            states=[Question(question="What is the capital of Italy?")])

## Instantiate the target AG with a target type. No instances are needed for zero shot transduction
target= AG(atype=Answer,  ## You can choose between "openai", "watsonx", "gemini", "vllm_crewai"
           verbose_agent=True)   ## set verbose to true to see the internal agents log

# Execute logical transduction by using the << operator between source and target AG
answer = await (target << source) ## Note that << operator is asyncronus and the results should be awaited

# Print the results of the transduction
print(answer.pretty_print()) ## Note that confidence is a float number, while other fields are strings

2025-09-12 11:35:57.357 | DEBUG    | agentics.core.llm_connections:get_llm_provider:29 - No LLM provider specified. Using the first available provider.
2025-09-12 11:35:57.357 | DEBUG    | agentics.core.llm_connections:get_llm_provider:31 - Available LLM providers: ['watsonx', 'gemini', 'openai']. Using 'watsonx'
2025-09-12 11:35:57.358 | DEBUG    | agentics.core.agentics:__lshift__:518 - Executing task: Generate an object of the specified type from the following input.
0 states will be transduced


TypeError: 'NoneType' object is not iterable

In agentics, lists of strings can be used as sources instead of AG. Those are provided as input for the transduction.

In [3]:
answer = await (AG(atype=Answer) << ["Where is Paris?"])
print(answer.pretty_print())

2025-09-11 11:26:18.000 | DEBUG    | agentics.core.agentics:__lshift__:545 - Transduction from input texts ['Where is Paris?'] to <class 'pydantic._internal._model_construction.ModelMetaclass'> in progress. This might take a while
2025-09-11 11:26:18.001 | DEBUG    | agentics.core.agentics:__lshift__:612 - transducer class: <class 'agentics.abstractions.pydantic_transducer.PydanticTransducerCrewAI'>
2025-09-11 11:26:19.778 | DEBUG    | agentics.core.agentics:__lshift__:648 - Processed 1 states in 1.7765769958496094 seconds
2025-09-11 11:26:19.779 | DEBUG    | agentics.core.agentics:__lshift__:700 - 1 states processed in 0.08882884979248047 seconds average per state ...


Atype : <class '__main__.Answer'>
answer: Paris is in France
justification: The question asks for the location of Paris, which is a well-known
  city in France.
confidence: 0.99




## Asyncronous Transduction

When the source AG has more than one state, the << operator perform asyncronous transfuction for each state to the target type. Transductions are executed in batches.

In [4]:
target = AG(atype=Answer,
            batch_size = 4, # This will process 2 questions per batch
            verbose_transduction = True, # Set to verbose to see transduction timings and other logs
            transduction_logs_path = "/tmp/answers.jsonl") # Optionally write longs of transductions on the specified path
questions= ["Where is Paris?",
            "Who is Alberto Sordi",
            "When will climate change be irreverible?",
            "Who is the best Jeopardy player?"]

answers = await (target << questions)

print(answer.pretty_print())

target.batch_size = 4  ## Processing 4 questions in parallel often results in
                                    ## lower execution time
                                    ## if supported by LLM backend

answers = await (target << questions)

2025-09-11 11:26:22.614 | DEBUG    | agentics.core.agentics:__lshift__:545 - Transduction from input texts ['Where is Paris?', 'Who is Alberto Sordi', 'When will climate change be irreverible?', 'Who is the best Jeopardy player?'] to <class 'pydantic._internal._model_construction.ModelMetaclass'> in progress. This might take a while
2025-09-11 11:26:22.614 | DEBUG    | agentics.core.agentics:__lshift__:612 - transducer class: <class 'agentics.abstractions.pydantic_transducer.PydanticTransducerCrewAI'>
2025-09-11 11:26:27.895 | DEBUG    | agentics.core.agentics:__lshift__:648 - Processed 4 states in 5.280547857284546 seconds
2025-09-11 11:26:27.896 | DEBUG    | agentics.core.agentics:__lshift__:700 - 4 states processed in 1.3201369643211365 seconds average per state ...
2025-09-11 11:26:27.897 | DEBUG    | agentics.core.agentics:__lshift__:545 - Transduction from input texts ['Where is Paris?', 'Who is Alberto Sordi', 'When will climate change be irreverible?', 'Who is the best Jeopardy

Atype : <class '__main__.Answer'>
answer: Paris is in France
justification: The question asks for the location of Paris, which is a well-known
  city in France.
confidence: 0.99




2025-09-11 11:26:31.887 | DEBUG    | agentics.core.agentics:__lshift__:648 - Processed 4 states in 3.988463878631592 seconds
2025-09-11 11:26:31.888 | DEBUG    | agentics.core.agentics:__lshift__:700 - 4 states processed in 0.997115969657898 seconds average per state ...


## Self Transduction

Self transduction is a method of AGs that enables async execution of transductions between slots of the same object.

In [5]:
from pydantic import BaseModel, Field
from typing import Optional
from pathlib import Path
## Define the Pydantic type
class Movie(BaseModel):
    movie_name: Optional[str] = None ## Note that fields name should match the column name in the input csv
    genre: Optional[str] = None
    description: Optional[str]=None
    tweet: Optional[str]=Field(None,description="Generate a Tweet to advertise the movie")
base = Path(CURRENT_PATH)
movies = AG.from_csv( base / "data/movies.csv", atype=Movie, max_rows=20) ## Load the input data from a csv file
movies.verbose_transduction = True
movies.llm=get_llm_provider("watsonx") ## You can choose between "openai", "watsonx", "gemini", "vllm_crewai"

movies_with_tweets = await movies.self_transduction(

    ["movie_name", "genre", "description"], ## source fields
    ["tweet"],                              ## target fields

    ## Note that instruction are only needed when the  relation between source and target type
    # is not innediately clear and need to be further specified or disambiguated.
    instructions="Generate a tweet to advertise the release of the input movie")

print(movies_with_tweets.pretty_print())

2025-09-11 11:26:33.257 | DEBUG    | agentics.core.agentics:from_csv:312 - Importing Agentics of type Movie from CSV data/movies.csv
2025-09-11 11:26:33.259 | DEBUG    | agentics.core.llm_connections:get_llm_provider:42 - Using specified LLM provider: watsonx
2025-09-11 11:26:33.260 | DEBUG    | agentics.core.agentics:__lshift__:518 - Executing task: Generate a tweet to advertise the release of the input movie
20 states will be transduced
2025-09-11 11:26:33.260 | DEBUG    | agentics.core.agentics:__lshift__:612 - transducer class: <class 'agentics.abstractions.pydantic_transducer.PydanticTransducerCrewAI'>
2025-09-11 11:26:39.106 | DEBUG    | agentics.core.agentics:__lshift__:648 - Processed 20 states in 5.845726013183594 seconds
2025-09-11 11:26:39.107 | DEBUG    | agentics.core.agentics:__lshift__:700 - 20 states processed in 0.2922863006591797 seconds average per state ...


Atype : <class '__main__.Movie'>
movie_name: The Shawshank Redemption
genre: Drama, Crime
description: Imprisoned in the 1940s for the double murder of his wife and her lover,
  upstanding banker Andy Dufresne begins a new life at the Shawshank prison, where
  he puts his accounting skills to work for an amoral warden. During his long stretch
  in prison, Dufresne comes to be admired by the other inmates -- including an older
  prisoner named Red -- for his integrity and unquenchable sense of hope.
tweet: 'Get ready to experience hope and redemption! ''The Shawshank Redemption'',
  a gripping Drama and Crime movie, is now out! Imprisoned for a double murder, Andy
  Dufresne''s journey to freedom will leave you inspired. Don''t miss this classic!
  #TheShawshankRedemption #NewRelease'

movie_name: The Godfather
genre: Drama, Crime
description: Spanning the years 1945 to 1955, a chronicle of the fictional Italian-American
  Corleone crime family. When organized crime family patriarch, Vi

As an alternative, self transduction can be encoded using logical transduction algebra which uses the AG() notation to rebind the original data into AGs of the requested subtypes. Learn more about atype manipulation in agentics [here](link)

In [6]:
movies = AG.from_csv( base / "data/movies.csv", atype=Movie, max_rows=20)
tweets = await (AG(atype=movies("tweet").atype,
                   instructions="Generate a tweet to advertise the release of the input movie")
                << movies("movie_name","genre","description" ))
print(tweets.pretty_print())    ## Note that differently from self transduction the output tweets
                                ##has only the tweet field, whereas self transduction preserves
                                ## the original source data

2025-09-11 11:26:45.851 | DEBUG    | agentics.core.agentics:from_csv:312 - Importing Agentics of type Movie from CSV data/movies.csv
2025-09-11 11:26:45.854 | DEBUG    | agentics.core.agentics:__lshift__:518 - Executing task: Generate a tweet to advertise the release of the input movie
20 states will be transduced
2025-09-11 11:26:45.855 | DEBUG    | agentics.core.agentics:__lshift__:612 - transducer class: <class 'agentics.abstractions.pydantic_transducer.PydanticTransducerCrewAI'>
2025-09-11 11:26:50.763 | DEBUG    | agentics.core.agentics:__lshift__:648 - Processed 20 states in 4.907993793487549 seconds
2025-09-11 11:26:50.764 | DEBUG    | agentics.core.agentics:__lshift__:700 - 20 states processed in 0.24539968967437745 seconds average per state ...


Atype : <class 'agentics.core.agentics.tweet'>
tweet: 'Get ready to experience hope and redemption! ''The Shawshank Redemption'',
  a gripping Drama and Crime movie, is now out! Imprisoned for a crime he didn''t
  commit, Andy Dufresne''s story of integrity and hope will leave you inspired. Don''t
  miss it! #TheShawshankRedemption #NewRelease'

tweet: 'Get ready for a cinematic masterpiece! ''The Godfather'', a gripping Drama
  and Crime film, is now out! Spanning 1945 to 1955, it chronicles the Italian-American
  Corleone crime family''s bloody revenge saga. Don''t miss it! #TheGodfather #Drama
  #Crime'

tweet: 'Get ready for a cinematic masterpiece! ''The Godfather Part II'' is now out!
  This Drama, Crime film follows the Corleone crime family''s saga as they expand
  their business into Las Vegas, Hollywood, and Cuba. Don''t miss out on the thrilling
  story of the Corleone family''s rise to power! #TheGodfatherPartII #Drama #Crime'

tweet: 'Get ready to witness a powerful true s

## Few Shots Transduction

Few shots examples can be provided for transduction by adding instances of the target instances in correspondance to their sources . Those will be used by the LLM to infer by analogy all the Null instances of the target type.


In [ ]:
movies = AG.from_csv(base / "data/movies.csv", max_rows=20)
## Note that obly the first 9 movies have categories
for i, movie in enumerate(movies): print(f"{i}: {movie.genre}")
## predicting new genre from given examples
all_genres=await (movies("genre")<<movies("movie_name",  "description"))
print(all_genres.pretty_print())
## few shots examples can also be used in self transfuction
movies_with_genre = await movies.self_transduction(["movie_name",  "description"],["genre"])
print(movies_with_genre.pretty_print())

2025-09-11 11:25:13.649 | DEBUG    | agentics.core.agentics:__lshift__:518 - Executing task: Generate an object of the specified type from the following input.
20 states will be transduced
2025-09-11 11:25:13.649 | DEBUG    | agentics.core.agentics:__lshift__:612 - transducer class: <class 'agentics.abstractions.pydantic_transducer.PydanticTransducerCrewAI'>


0: Drama, Crime
1: Drama, Crime
2: Drama, Crime
3: Drama, History, War
4: Drama
5: Animation, Family, Fantasy
6: Drama, Action, Crime, Thriller
7: Comedy, Drama, Romance
8: Fantasy, Drama, Crime
9: 
10: 
11: 
12: 
13: 
14: 
15: 
16: 
17: 
18: 
19: 
